In [5]:
import anndata as ad
import numpy as np
from image_generators.SOM import SOM

In [6]:
adata = ad.read_h5ad("/nobackup/scmpht/CNN-seq/data/model_ready/dummy_GTEx.h5ad")
adata

AnnData object with n_obs × n_vars = 1962 × 1539
    obs: 'SMTS', 'SMTSD', 'SMNABTCH', 'SMNABTCHT', 'SMNABTCHD', 'SMGEBTCH', 'SMGEBTCHD', 'SMGEBTCHT', 'SMCENTER'
    var: 'Symbol'

In [ ]:
# Filter for training data

In [ ]:
# Z-score inputs

In [ ]:
# Create a data loader and run in batches

In [ ]:
# Training grid
m = 32
n = 32
dim = training_colours.shape[1]

# Instatiate SOM
som = SOM(m, n, dim, decay=0.95, alpha=0.3)


max_iter = 200
alpha = 0.3
best_objective = np.inf
patience = 0
tolerance = 0.01 * som.get_avg_bmu_dist(training_colours).numpy() # 1% of initial distances

def objective_function(distance, changes, alpha=1, beta=1):
    return distance*alpha + changes*beta


best_objectives = np.array([])
avg_distances = np.array([])
num_changes = np.array([])

bmu_list = som.get_bmus(training_colours)

for iter_no in range(max_iter):
    
    # 1) Perform the training update for this iteration
    for i in range(len(training_colours)):
        som(training_colours[i], iter_no, GIF=GIF)
    
    # 2) Compute new average distance
    this_avg_dist = som.get_avg_bmu_dist(training_colours).numpy()
    avg_distances = np.append(avg_distances, this_avg_dist)

    # 3) Compute new changes
    bmu_list, changes = som.get_bmu_changes(training_colours, bmu_list)
    num_changes = np.append(num_changes, changes)

    # 4) Now compute objective using the *freshly computed* values
    objective = objective_function(this_avg_dist, changes, alpha=1, beta=0.001)

    # 5) Check patience
    if objective < best_objective - tolerance:
        best_objective = objective
        patience = 0
    else:
        patience += 1

    best_objectives = np.append(best_objectives, best_objective)
    if patience == 10:
        break

In [ ]:
fig, ax1 = plt.subplots()

# Plot avg_distances on the first axis (separate y-axis)
p1, = ax1.plot(avg_distances, c="#46e0bf", label="Distances")
ax1.set_ylabel('Objective')
ax1.tick_params(axis='y')
ax1.set_xlabel('Epochs')



# Add a second y-axis for num_changes and best_objective
ax2 = ax1.twinx()

# Plot num_changes and best_objective on the same axis
p2, = ax2.plot(num_changes, c="#8c5dd0", label="Changes", linestyle='-')
p3, = ax1.plot(best_objectives, c="#ff6347", label="Objective", linestyle='--')

# Set labels for ax2
ax2.set_ylabel('Changes')
ax2.tick_params(axis='y')

# Add a combined legend for all metrics
lines = [p1, p2, p3]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right')

# Add a title
plt.title("Metrics Over Epochs")
plt.show()